# **Enhanced_Face_Resolution via GFPGAN Model**

In [ ]:
!pip install basicsr facexlib
!pip install -q gfpgan
!pip install opencv-python matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 k

In [ ]:
# Clone the GFPGAN repo to access the models
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -P experiments/pretrained_models

Cloning into 'GFPGAN'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 527 (delta 170), reused 154 (delta 154), pack-reused 315 (from 2)
Receiving objects: 100% (527/527), 5.38 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (281/281), done.
/content/GFPGAN
--2025-04-16 07:37:35--  https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/349321229/e9847322-b8b1-4ec2-9620-5146eb8a9e4b?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250416%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250416T073736Z&X-Amz-Expires=300&X-Amz-Signature=6172bd590af3c75538441fc01df88bdd624cbdc56bc4546860fc9

In [ ]:
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import shutil
from basicsr.archs.rrdbnet_arch import RRDBNet
from gfpgan import GFPGANer
from google.colab.patches import cv2_imshow
import torch
import zipfile


## **Image Super-Resolution: Enhancing 28×28 Images**


In [ ]:
def enhance_dataset_with_gfpgan(input_dataset_dir, output_dataset_dir):
    """
    Process an entire emotion dataset using GFPGAN

    Args:
        input_dataset_dir (str): Path to the input dataset directory containing emotion folders
        output_dataset_dir (str): Path to save the enhanced dataset with the same structure
    """

    os.makedirs(output_dataset_dir, exist_ok=True) # Check if the output directory exists


    model = GFPGANer(
        model_path='experiments/pretrained_models/GFPGANv1.4.pth',
        upscale=8,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=None
    )


    emotion_folders = [f for f in os.listdir(input_dataset_dir)   # Get all emotion folders
                       if os.path.isdir(os.path.join(input_dataset_dir, f))]

    # Process each emotion folder
    for emotion in emotion_folders:
        input_emotion_path = os.path.join(input_dataset_dir, emotion)
        output_emotion_path = os.path.join(output_dataset_dir, emotion)


        os.makedirs(output_emotion_path, exist_ok=True)

        # Get all images in the emotion folder
        image_files = [f for f in os.listdir(input_emotion_path)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

        print(f"Processing {len(image_files)} images in '{emotion}' folder...")


        for img_file in image_files:
            input_img_path = os.path.join(input_emotion_path, img_file)

            output_img_path = os.path.join(output_emotion_path, img_file)


            img = cv2.imread(input_img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"Could not read image: {input_img_path}, skipping...")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Enhance face
            _, _, restored_img = model.enhance(
                img_rgb,
                has_aligned=False,
                only_center_face=False,
                paste_back=True
            )

            cv2_restored = cv2.cvtColor(restored_img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_img_path, cv2_restored)

            print(f"  Enhanced: {img_file}")

    print(f"\nEnhancement complete! Enhanced dataset saved to: {output_dataset_dir}")


zip_path = '/content/KEDF_downsample_28x28.zip'
print(f"Extracting {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./")

input_dir = "./KEDF_downsample_28x28"
output_dir = "./KEDF_downsample_28x28_Enhanced"

enhance_dataset_with_gfpgan(input_dir, output_dir)

output_zip = "KEDF_DataSet_Enhanced.zip"
print(f"Creating zip file of enhanced dataset: {output_zip}")
shutil.make_archive(os.path.splitext(output_zip)[0], 'zip', output_dir)

from google.colab import files
files.download(output_zip)

Extracting /content/KEDF_downsample_28x28.zip...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/GFPGAN/gfpgan/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 142MB/s]


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/GFPGAN/gfpgan/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 129MB/s]


Streaming output truncated to the last 5000 lines.
  Enhanced: AM24AFFL.JPG
  Enhanced: AM24AFHR.JPG
  Enhanced: AM24NEFL.JPG
  Enhanced: AM24ANHR.JPG
  Enhanced: AM24NEFR.JPG
  Enhanced: AM24SAS.JPG
  Enhanced: AM24AFFR.JPG
  Enhanced: AM24SAHL.JPG
  Enhanced: AM24AFS.JPG
  Enhanced: AM24NEHR.JPG
  Enhanced: AM24AFHL.JPG
  Enhanced: AM24SUFL.JPG
  Enhanced: AM24HAHL.JPG
  Enhanced: AM24ANFL.JPG
  Enhanced: AM24HAFR.JPG
  Enhanced: AM24SAFL.JPG
  Enhanced: AM24SAFR.JPG
  Enhanced: AM24ANFR.JPG
  Enhanced: AM24NES.JPG
  Enhanced: AM24HAHR.JPG
  Enhanced: AM24NEHL.JPG
  Enhanced: AM24DIS.JPG
  Enhanced: AM24DIFL.JPG
  Enhanced: AM24SUFR.JPG
  Enhanced: AM24DIFR.JPG
  Enhanced: AM24SUHR.JPG
  Enhanced: AM24ANS.JPG
  Enhanced: AM24HAFL.JPG
  Enhanced: AM24HAS.JPG
Processing 35 images in 'BF09' folder...
  Enhanced: BF09SAFR.JPG
  Enhanced: BF09AFS.JPG
  Enhanced: BF09ANHR.JPG
  Enhanced: BF09NES.JPG
  Enhanced: BF09HAFR.JPG
  Enhanced: BF09DIFL.JPG
  Enhanced: BF09DIFR.JPG
  Enhanced: BF09

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##  **Image Super-Resolution: Enhancing 56×56 Images**

In [ ]:
def enhance_dataset_with_gfpgan(input_dataset_dir, output_dataset_dir):
    """
    Process an entire emotion dataset using GFPGAN

    Args:
        input_dataset_dir (str): Path to the input dataset directory containing emotion folders
        output_dataset_dir (str): Path to save the enhanced dataset with the same structure
    """

    os.makedirs(output_dataset_dir, exist_ok=True)

    model = GFPGANer(
        model_path='experiments/pretrained_models/GFPGANv1.4.pth',
        upscale=4,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=None
    )

    emotion_folders = [f for f in os.listdir(input_dataset_dir)
                       if os.path.isdir(os.path.join(input_dataset_dir, f))]


    for emotion in emotion_folders:
        input_emotion_path = os.path.join(input_dataset_dir, emotion)
        output_emotion_path = os.path.join(output_dataset_dir, emotion)


        os.makedirs(output_emotion_path, exist_ok=True)

        image_files = [f for f in os.listdir(input_emotion_path)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

        print(f"Processing {len(image_files)} images in '{emotion}' folder...")

        for img_file in image_files:
            input_img_path = os.path.join(input_emotion_path, img_file)

            output_img_path = os.path.join(output_emotion_path, img_file)

            img = cv2.imread(input_img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"Could not read image: {input_img_path}, skipping...")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            _, _, restored_img = model.enhance(
                img_rgb,
                has_aligned=False,
                only_center_face=False,
                paste_back=True
            )

            cv2_restored = cv2.cvtColor(restored_img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_img_path, cv2_restored)

            print(f"  Enhanced: {img_file}")

    print(f"\nEnhancement complete! Enhanced dataset saved to: {output_dataset_dir}")

zip_path = '/content/KEDF_downsample_56x56.zip'
print(f"Extracting {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./")


input_dir = "./KEDF_downsample_56x56"
output_dir = "./KEDF_downsample_56x56_Enhanced"

enhance_dataset_with_gfpgan(input_dir, output_dir)

output_zip = "KEDF_DataSet_Enhanced.zip"
print(f"Creating zip file of enhanced dataset: {output_zip}")
shutil.make_archive(os.path.splitext(output_zip)[0], 'zip', output_dir)

from google.colab import files
files.download(output_zip)

Extracting /content/KEDF_downsample_56x56.zip...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/GFPGAN/gfpgan/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 477MB/s] 


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/GFPGAN/gfpgan/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 380MB/s]


Streaming output truncated to the last 5000 lines.
  Enhanced: AM24AFFL.JPG
  Enhanced: AM24AFHR.JPG
  Enhanced: AM24NEFL.JPG
  Enhanced: AM24ANHR.JPG
  Enhanced: AM24NEFR.JPG
  Enhanced: AM24SAS.JPG
  Enhanced: AM24AFFR.JPG
  Enhanced: AM24SAHL.JPG
  Enhanced: AM24AFS.JPG
  Enhanced: AM24NEHR.JPG
  Enhanced: AM24AFHL.JPG
  Enhanced: AM24SUFL.JPG
  Enhanced: AM24HAHL.JPG
  Enhanced: AM24ANFL.JPG
  Enhanced: AM24HAFR.JPG
  Enhanced: AM24SAFL.JPG
  Enhanced: AM24SAFR.JPG
  Enhanced: AM24ANFR.JPG
  Enhanced: AM24NES.JPG
  Enhanced: AM24HAHR.JPG
  Enhanced: AM24NEHL.JPG
  Enhanced: AM24DIS.JPG
  Enhanced: AM24DIFL.JPG
  Enhanced: AM24SUFR.JPG
  Enhanced: AM24DIFR.JPG
  Enhanced: AM24SUHR.JPG
  Enhanced: AM24ANS.JPG
  Enhanced: AM24HAFL.JPG
  Enhanced: AM24HAS.JPG
Processing 35 images in 'BF09' folder...
  Enhanced: BF09SAFR.JPG
  Enhanced: BF09AFS.JPG
  Enhanced: BF09ANHR.JPG
  Enhanced: BF09NES.JPG
  Enhanced: BF09HAFR.JPG
  Enhanced: BF09DIFL.JPG
  Enhanced: BF09DIFR.JPG
  Enhanced: BF09

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##  **Image Super-Resolution: Enhancing 112×112 Images**

In [ ]:
def enhance_dataset_with_gfpgan(input_dataset_dir, output_dataset_dir):
    """
    Process an entire emotion dataset using GFPGAN

    Args:
        input_dataset_dir (str): Path to the input dataset directory containing emotion folders
        output_dataset_dir (str): Path to save the enhanced dataset with the same structure
    """

    os.makedirs(output_dataset_dir, exist_ok=True)


    model = GFPGANer(
        model_path='experiments/pretrained_models/GFPGANv1.4.pth',
        upscale=2,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=None
    )


    emotion_folders = [f for f in os.listdir(input_dataset_dir)
                       if os.path.isdir(os.path.join(input_dataset_dir, f))]

    for emotion in emotion_folders:
        input_emotion_path = os.path.join(input_dataset_dir, emotion)
        output_emotion_path = os.path.join(output_dataset_dir, emotion)

        os.makedirs(output_emotion_path, exist_ok=True)

        image_files = [f for f in os.listdir(input_emotion_path)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

        print(f"Processing {len(image_files)} images in '{emotion}' folder...")

        for img_file in image_files:
            input_img_path = os.path.join(input_emotion_path, img_file)

            output_img_path = os.path.join(output_emotion_path, img_file)


            img = cv2.imread(input_img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"Could not read image: {input_img_path}, skipping...")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            _, _, restored_img = model.enhance(
                img_rgb,
                has_aligned=False,
                only_center_face=False,
                paste_back=True
            )

            cv2_restored = cv2.cvtColor(restored_img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_img_path, cv2_restored)

            print(f"  Enhanced: {img_file}")

    print(f"\nEnhancement complete! Enhanced dataset saved to: {output_dataset_dir}")


zip_path = '/content/KEDF_downsample_112x112.zip'
print(f"Extracting {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./")

input_dir = "./KEDF_downsample_112x112"
output_dir = "./KEDF_downsample_112x112_Enhanced"

enhance_dataset_with_gfpgan(input_dir, output_dir)

output_zip = "KEDF_DataSet_Enhanced.zip"
print(f"Creating zip file of enhanced dataset: {output_zip}")
shutil.make_archive(os.path.splitext(output_zip)[0], 'zip', output_dir)

from google.colab import files
files.download(output_zip)

Streaming output truncated to the last 5000 lines.
  Enhanced: AM24AFFL.JPG
  Enhanced: AM24AFHR.JPG
  Enhanced: AM24NEFL.JPG
  Enhanced: AM24ANHR.JPG
  Enhanced: AM24NEFR.JPG
  Enhanced: AM24SAS.JPG
  Enhanced: AM24AFFR.JPG
  Enhanced: AM24SAHL.JPG
  Enhanced: AM24AFS.JPG
  Enhanced: AM24NEHR.JPG
  Enhanced: AM24AFHL.JPG
  Enhanced: AM24SUFL.JPG
  Enhanced: AM24HAHL.JPG
  Enhanced: AM24ANFL.JPG
  Enhanced: AM24HAFR.JPG
  Enhanced: AM24SAFL.JPG
  Enhanced: AM24SAFR.JPG
  Enhanced: AM24ANFR.JPG
  Enhanced: AM24NES.JPG
  Enhanced: AM24HAHR.JPG
  Enhanced: AM24NEHL.JPG
  Enhanced: AM24DIS.JPG
  Enhanced: AM24DIFL.JPG
  Enhanced: AM24SUFR.JPG
  Enhanced: AM24DIFR.JPG
  Enhanced: AM24SUHR.JPG
  Enhanced: AM24ANS.JPG
  Enhanced: AM24HAFL.JPG
  Enhanced: AM24HAS.JPG
Processing 35 images in 'BF09' folder...
  Enhanced: BF09SAFR.JPG
  Enhanced: BF09AFS.JPG
  Enhanced: BF09ANHR.JPG
  Enhanced: BF09NES.JPG
  Enhanced: BF09HAFR.JPG
  Enhanced: BF09DIFL.JPG
  Enhanced: BF09DIFR.JPG
  Enhanced: BF09

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Enhanced_Face_Resolution via CodeFormer Model**

In [ ]:
!pip install facexlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.7 MB/s eta 0:00:00

In [ ]:
# Clone CodeFormer and enter the CodeFormer folder
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd CodeFormer

!pip install -r requirements.txt # Set up the environment

!python basicsr/setup.py develop # Install basicsr

!python scripts/download_pretrained_models.py facelib #pre-trained model
!python scripts/download_pretrained_models.py CodeFormer


/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 614 (delta 202), reused 176 (delta 176), pack-reused 322 (from 3)
Receiving objects: 100% (614/614), 17.31 MiB | 7.68 MiB/s, done.
Resolving deltas: 100% (296/296), done.
/content/CodeFormer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.3 MB/s eta 0:00:00
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        **************************************************************

In [ ]:
import os
import cv2
import torch
import numpy as np
import zipfile
import shutil
from facexlib.utils.face_restoration_helper import FaceRestoreHelper
from torchvision.transforms.functional import normalize
from basicsr.utils import img2tensor, tensor2img
from google.colab import files
import glob
import torch
from basicsr.utils.registry import ARCH_REGISTRY

## **Image Super-Resolution: Enhancing 28×28 Images**

In [ ]:

def load_codeformer_model(device='cuda', fidelity=0.8):
    """Load CodeFormer model once into memory."""
    print("Loading CodeFormer model...")

    # CodeFormer model
    codeformer = ARCH_REGISTRY.get('CodeFormer')(
    dim_embd=512,
    codebook_size=1024,
    n_head=8,
    n_layers=9,
    connect_list=['32', '64', '128', '256']
    ).to(device)



    ckpt_path = './weights/CodeFormer/codeformer.pth' # pretrained weights
    checkpoint = torch.load(ckpt_path)['params_ema']
    codeformer.load_state_dict(checkpoint)
    codeformer.eval()

    return codeformer

def process_image(img_path, output_path, codeformer, face_helper, w=0.8):
    """Process a single image using an already loaded model."""

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Warning: Could not read image {img_path}")
        return False


    face_helper.clean_all()     # Detect and extract faces
    face_helper.read_image(img)
    face_helper.get_face_landmarks_5(only_center_face=False, resize=640, eye_dist_threshold=5)
    face_helper.align_warp_face()

    # Restore faces
    for idx, cropped_face in enumerate(face_helper.cropped_faces):
        # Prepare face image
        cropped_face_t = img2tensor(cropped_face / 255., bgr2rgb=True, float32=True)
        normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
        cropped_face_t = cropped_face_t.unsqueeze(0).to('cuda')

        try:
            with torch.no_grad():
                output = codeformer(cropped_face_t, w=w, adain=True)[0]
                restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
            face_helper.add_restored_face(restored_face)
        except Exception as e:
            print(f"Error processing face: {e}")
            face_helper.add_restored_face(cropped_face)

    # Paste faces back to image
    face_helper.get_inverse_affine(None)
    restored_img = face_helper.paste_faces_to_input_image()

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, restored_img)

    return True

def batch_process_dataset(input_dir, output_dir, w=0.8, upscale=8, face_upsample=True):
    """Batch process entire dataset using a cached model instance."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    codeformer = load_codeformer_model(device, w)

    face_helper = FaceRestoreHelper(
        upscale,
        face_size=512,
        crop_ratio=(1, 1),
        det_model='retinaface_resnet50',
        save_ext='png',
        use_parse=True,
        device=device
    )


    emotion_folders = [f for f in os.listdir(input_dir)
                      if os.path.isdir(os.path.join(input_dir, f))]

    total_images = 0
    for emotion in emotion_folders:
        input_emotion_path = os.path.join(input_dir, emotion) # Get all emotion folders
        output_emotion_path = os.path.join(output_dir, emotion)
        os.makedirs(output_emotion_path, exist_ok=True)

        image_files = []
        for ext in ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.bmp', '.BMP']:
            image_files.extend(glob.glob(os.path.join(input_emotion_path, f"*{ext}")))

        if not image_files:
            print(f"No images found in {input_emotion_path}, skipping...")
            continue

        print(f"Processing {len(image_files)} images in '{emotion}' folder...")
        total_images += len(image_files)

        for i, img_file in enumerate(image_files):
            output_img_path = os.path.join(output_emotion_path, os.path.basename(img_file))


            if i % 5 == 0:    #print the process
                print(f"  Processing image {i+1}/{len(image_files)} in folder {emotion}")


            process_image(img_file, output_img_path, codeformer, face_helper, w)

        print(f"Finished processing {emotion} folder")

    print(f"\nProcessing complete! Total {total_images} images processed and saved to: {output_dir}")


zip_path = '/content/KEDF_downsample_28x28.zip'

if not os.path.exists(zip_path):
    print(f"Error: ZIP file {zip_path} not found. Please upload it first.")
    from google.colab import files
    print("Please upload your ZIP file:")
    uploaded = files.upload()
    zip_path = list(uploaded.keys())[0]

# Extract the file
print(f"Extracting {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

# Check if directory exists after extraction
input_dir = "/content/KEDF_downsample_28x28"
if not os.path.exists(input_dir):
    print(f"Directory {input_dir} not found after extraction.")

    extracted_dirs = [d for d in os.listdir("/content") if os.path.isdir(os.path.join("/content", d))]
    print(f"Existing directories in /content: {extracted_dirs}")

    # Try to find a matching directory
    potential_dir = None
    for d in extracted_dirs:
        if "kedf" in d.lower() or "downsample" in d.lower():
            potential_dir = os.path.join("/content", d)
            break

    if potential_dir:
        print(f"Found potential directory: {potential_dir}")
        input_dir = potential_dir
    else:
        print("No matching directory found. Please check the dataset directory name.")
        # Ask user to input the correct directory path
        input_dir = input("Enter the full path to the dataset directory: ")


output_dir = "/content/KEDF_downsample_28x28_CodeFormer_Enhanced"
os.makedirs(output_dir, exist_ok=True)

# Set CodeFormer parameters

CODEFORMER_FIDELITY = 0.8  # Balance quality (lower) and fidelity (higher)
UPSCALE_FACTOR = 8  # Upscaling factor
FACE_UPSAMPLE = True  # Whether to upsample restored faces

# Run the batch processing function
batch_process_dataset(
    input_dir=input_dir,
    output_dir=output_dir,
    w=CODEFORMER_FIDELITY,
    upscale=UPSCALE_FACTOR,
    face_upsample=FACE_UPSAMPLE
)


%cd /content
output_zip = "KEDF_DataSet_CodeFormer_Enhanced.zip"
print(f"Creating ZIP file of enhanced dataset: {output_zip}")
shutil.make_archive(os.path.splitext(output_zip)[0], 'zip', output_dir)

files.download(output_zip)

Extracting /content/KEDF_downsample_28x28.zip...
Loading CodeFormer model...


<ipython-input-4-8181f6619d49>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path)['params_ema']


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /usr/local/lib/python3.11/dist-packages/facexlib/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 491MB/s] 
/usr/local/lib/python3.11/dist-packages/facexlib/detection/__init__.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  l

Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /usr/local/lib/python3.11/dist-packages/facexlib/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 422MB/s]
/usr/local/lib/python3.11/dist-packages/facexlib/parsing/__init__.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lo

Processing 35 images in 'AF23' folder...
  Processing image 1/35 in folder AF23
  Processing image 6/35 in folder AF23
  Processing image 11/35 in folder AF23
  Processing image 16/35 in folder AF23
  Processing image 21/35 in folder AF23
  Processing image 26/35 in folder AF23
  Processing image 31/35 in folder AF23
Finished processing AF23 folder
Processing 35 images in 'AM24' folder...
  Processing image 1/35 in folder AM24
  Processing image 6/35 in folder AM24
  Processing image 11/35 in folder AM24
  Processing image 16/35 in folder AM24
  Processing image 21/35 in folder AM24
  Processing image 26/35 in folder AM24
  Processing image 31/35 in folder AM24
Finished processing AM24 folder
Processing 35 images in 'BF09' folder...
  Processing image 1/35 in folder BF09
  Processing image 6/35 in folder BF09
  Processing image 11/35 in folder BF09
  Processing image 16/35 in folder BF09
  Processing image 21/35 in folder BF09
  Processing image 26/35 in folder BF09
  Processing image 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Image Super-Resolution: Enhancing 56×56 Images**

In [ ]:
%cd /content/CodeFormer

/content/CodeFormer


In [ ]:

def load_codeformer_model(device='cuda', fidelity=0.8):
    """Load CodeFormer model once into memory."""
    print("Loading CodeFormer model...")


    codeformer = ARCH_REGISTRY.get('CodeFormer')(
    dim_embd=512,
    codebook_size=1024,
    n_head=8,
    n_layers=9,
    connect_list=['32', '64', '128', '256']
    ).to(device)



    ckpt_path = './weights/CodeFormer/codeformer.pth'
    checkpoint = torch.load(ckpt_path)['params_ema']
    codeformer.load_state_dict(checkpoint)
    codeformer.eval()

    return codeformer

def process_image(img_path, output_path, codeformer, face_helper, w=0.8):
    """Process a single image using an already loaded model."""
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Warning: Could not read image {img_path}")
        return False

    face_helper.clean_all()
    face_helper.read_image(img)
    face_helper.get_face_landmarks_5(only_center_face=False, resize=640, eye_dist_threshold=5)
    face_helper.align_warp_face()

    for idx, cropped_face in enumerate(face_helper.cropped_faces):
        cropped_face_t = img2tensor(cropped_face / 255., bgr2rgb=True, float32=True)
        normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
        cropped_face_t = cropped_face_t.unsqueeze(0).to('cuda')

        try:
            with torch.no_grad():
                output = codeformer(cropped_face_t, w=w, adain=True)[0]
                restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
            face_helper.add_restored_face(restored_face)
        except Exception as e:
            print(f"Error processing face: {e}")
            face_helper.add_restored_face(cropped_face)

    face_helper.get_inverse_affine(None)
    restored_img = face_helper.paste_faces_to_input_image()

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, restored_img)

    return True

def batch_process_dataset(input_dir, output_dir, w=0.8, upscale=8, face_upsample=True):
    """Batch process entire dataset using a cached model instance."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    codeformer = load_codeformer_model(device, w)

    face_helper = FaceRestoreHelper(
        upscale,
        face_size=512,
        crop_ratio=(1, 1),
        det_model='retinaface_resnet50',
        save_ext='png',
        use_parse=True,
        device=device
    )

    emotion_folders = [f for f in os.listdir(input_dir)
                      if os.path.isdir(os.path.join(input_dir, f))]

    total_images = 0
    for emotion in emotion_folders:
        input_emotion_path = os.path.join(input_dir, emotion)
        output_emotion_path = os.path.join(output_dir, emotion)
        os.makedirs(output_emotion_path, exist_ok=True)

        image_files = []
        for ext in ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.bmp', '.BMP']:
            image_files.extend(glob.glob(os.path.join(input_emotion_path, f"*{ext}")))

        if not image_files:
            print(f"No images found in {input_emotion_path}, skipping...")
            continue

        print(f"Processing {len(image_files)} images in '{emotion}' folder...")
        total_images += len(image_files)

        for i, img_file in enumerate(image_files):
            output_img_path = os.path.join(output_emotion_path, os.path.basename(img_file))

            if i % 5 == 0:
                print(f"  Processing image {i+1}/{len(image_files)} in folder {emotion}")

            process_image(img_file, output_img_path, codeformer, face_helper, w)

        print(f"Finished processing {emotion} folder")

    print(f"\nProcessing complete! Total {total_images} images processed and saved to: {output_dir}")

zip_path = '/content/KEDF_downsample_56x56.zip'

if not os.path.exists(zip_path):
    print(f"Error: ZIP file {zip_path} not found. Please upload it first.")
    from google.colab import files
    print("Please upload your ZIP file:")
    uploaded = files.upload()
    zip_path = list(uploaded.keys())[0]

print(f"Extracting {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

input_dir = "/content/KEDF_downsample_56x56"
if not os.path.exists(input_dir):
    print(f"Directory {input_dir} not found after extraction.")
    extracted_dirs = [d for d in os.listdir("/content") if os.path.isdir(os.path.join("/content", d))]
    print(f"Existing directories in /content: {extracted_dirs}")

    potential_dir = None
    for d in extracted_dirs:
        if "kedf" in d.lower() or "downsample" in d.lower():
            potential_dir = os.path.join("/content", d)
            break

    if potential_dir:
        print(f"Found potential directory: {potential_dir}")
        input_dir = potential_dir
    else:
        print("No matching directory found. Please check the dataset directory name.")
        input_dir = input("Enter the full path to the dataset directory: ")

output_dir = "/content/KEDF_downsample_56x56_CodeFormer_Enhanced"
os.makedirs(output_dir, exist_ok=True)


CODEFORMER_FIDELITY = 0.8
UPSCALE_FACTOR = 4
FACE_UPSAMPLE = True

batch_process_dataset(
    input_dir=input_dir,
    output_dir=output_dir,
    w=CODEFORMER_FIDELITY,
    upscale=UPSCALE_FACTOR,
    face_upsample=FACE_UPSAMPLE
)

%cd /content
output_zip = "KEDF_DataSet_CodeFormer_Enhanced.zip"
print(f"Creating ZIP file of enhanced dataset: {output_zip}")
shutil.make_archive(os.path.splitext(output_zip)[0], 'zip', output_dir)

# Provide download link
files.download(output_zip)

Extracting /content/KEDF_downsample_56x56.zip...
Loading CodeFormer model...


<ipython-input-12-1a7d2d5d8915>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path)['params_ema']


Processing 35 images in 'AF23' folder...
  Processing image 1/35 in folder AF23
  Processing image 6/35 in folder AF23
  Processing image 11/35 in folder AF23
  Processing image 16/35 in folder AF23
  Processing image 21/35 in folder AF23
  Processing image 26/35 in folder AF23
  Processing image 31/35 in folder AF23
Finished processing AF23 folder
Processing 35 images in 'AM24' folder...
  Processing image 1/35 in folder AM24
  Processing image 6/35 in folder AM24
  Processing image 11/35 in folder AM24
  Processing image 16/35 in folder AM24
  Processing image 21/35 in folder AM24
  Processing image 26/35 in folder AM24
  Processing image 31/35 in folder AM24
Finished processing AM24 folder
Processing 35 images in 'BF09' folder...
  Processing image 1/35 in folder BF09
  Processing image 6/35 in folder BF09
  Processing image 11/35 in folder BF09
  Processing image 16/35 in folder BF09
  Processing image 21/35 in folder BF09
  Processing image 26/35 in folder BF09
  Processing image 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Image Super-Resolution: Enhancing 112×112 Images**

In [ ]:
%cd /content/CodeFormer

/content/CodeFormer


In [ ]:

def load_codeformer_model(device='cuda', fidelity=0.8):
    """Load CodeFormer model once into memory."""
    print("Loading CodeFormer model...")

    codeformer = ARCH_REGISTRY.get('CodeFormer')(
    dim_embd=512,
    codebook_size=1024,
    n_head=8,
    n_layers=9,
    connect_list=['32', '64', '128', '256']
    ).to(device)


    ckpt_path = './weights/CodeFormer/codeformer.pth'
    checkpoint = torch.load(ckpt_path)['params_ema']
    codeformer.load_state_dict(checkpoint)
    codeformer.eval()

    return codeformer

def process_image(img_path, output_path, codeformer, face_helper, w=0.8):
    """Process a single image using an already loaded model."""
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Warning: Could not read image {img_path}")
        return False
    face_helper.clean_all()
    face_helper.read_image(img)
    face_helper.get_face_landmarks_5(only_center_face=False, resize=640, eye_dist_threshold=5)
    face_helper.align_warp_face()

    for idx, cropped_face in enumerate(face_helper.cropped_faces):
        cropped_face_t = img2tensor(cropped_face / 255., bgr2rgb=True, float32=True)
        normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
        cropped_face_t = cropped_face_t.unsqueeze(0).to('cuda')

        try:
            with torch.no_grad():
                output = codeformer(cropped_face_t, w=w, adain=True)[0]
                restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
            face_helper.add_restored_face(restored_face)
        except Exception as e:
            print(f"Error processing face: {e}")
            face_helper.add_restored_face(cropped_face)

    face_helper.get_inverse_affine(None)
    restored_img = face_helper.paste_faces_to_input_image()

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, restored_img)

    return True

def batch_process_dataset(input_dir, output_dir, w=0.8, upscale=8, face_upsample=True):
    """Batch process entire dataset using a cached model instance."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    codeformer = load_codeformer_model(device, w)

    face_helper = FaceRestoreHelper(
        upscale,
        face_size=512,
        crop_ratio=(1, 1),
        det_model='retinaface_resnet50',
        save_ext='png',
        use_parse=True,
        device=device
    )
    emotion_folders = [f for f in os.listdir(input_dir)
                      if os.path.isdir(os.path.join(input_dir, f))]

    total_images = 0
    for emotion in emotion_folders:
        input_emotion_path = os.path.join(input_dir, emotion)
        output_emotion_path = os.path.join(output_dir, emotion)
        os.makedirs(output_emotion_path, exist_ok=True)

        image_files = []
        for ext in ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.bmp', '.BMP']:
            image_files.extend(glob.glob(os.path.join(input_emotion_path, f"*{ext}")))

        if not image_files:
            print(f"No images found in {input_emotion_path}, skipping...")
            continue

        print(f"Processing {len(image_files)} images in '{emotion}' folder...")
        total_images += len(image_files)

        for i, img_file in enumerate(image_files):
            output_img_path = os.path.join(output_emotion_path, os.path.basename(img_file))

            if i % 5 == 0:
                print(f"  Processing image {i+1}/{len(image_files)} in folder {emotion}")

            process_image(img_file, output_img_path, codeformer, face_helper, w)

        print(f"Finished processing {emotion} folder")

    print(f"\nProcessing complete! Total {total_images} images processed and saved to: {output_dir}")

zip_path = '/content/KEDF_downsample_112x112.zip'

if not os.path.exists(zip_path):
    print(f"Error: ZIP file {zip_path} not found. Please upload it first.")
    from google.colab import files
    print("Please upload your ZIP file:")
    uploaded = files.upload()
    zip_path = list(uploaded.keys())[0]

print(f"Extracting {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

input_dir = "/content/KEDF_downsample_112x112"
if not os.path.exists(input_dir):
    print(f"Directory {input_dir} not found after extraction.")
    extracted_dirs = [d for d in os.listdir("/content") if os.path.isdir(os.path.join("/content", d))]
    print(f"Existing directories in /content: {extracted_dirs}")
    potential_dir = None
    for d in extracted_dirs:
        if "kedf" in d.lower() or "downsample" in d.lower():
            potential_dir = os.path.join("/content", d)
            break

    if potential_dir:
        print(f"Found potential directory: {potential_dir}")
        input_dir = potential_dir
    else:
        print("No matching directory found. Please check the dataset directory name.")
        input_dir = input("Enter the full path to the dataset directory: ")

output_dir = "/content/KEDF_downsample_112x112_CodeFormer_Enhanced"
os.makedirs(output_dir, exist_ok=True)


CODEFORMER_FIDELITY = 0.8
UPSCALE_FACTOR = 4
FACE_UPSAMPLE = True


batch_process_dataset(
    input_dir=input_dir,
    output_dir=output_dir,
    w=CODEFORMER_FIDELITY,
    upscale=UPSCALE_FACTOR,
    face_upsample=FACE_UPSAMPLE
)


%cd /content
output_zip = "KEDF_DataSet_CodeFormer_Enhanced.zip"
print(f"Creating ZIP file of enhanced dataset: {output_zip}")
shutil.make_archive(os.path.splitext(output_zip)[0], 'zip', output_dir)


files.download(output_zip)

Extracting /content/KEDF_downsample_112x112.zip...
Loading CodeFormer model...


<ipython-input-15-da367047ee49>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path)['params_ema']


Processing 35 images in 'AF23' folder...
  Processing image 1/35 in folder AF23
  Processing image 6/35 in folder AF23
  Processing image 11/35 in folder AF23
  Processing image 16/35 in folder AF23
  Processing image 21/35 in folder AF23
  Processing image 26/35 in folder AF23
  Processing image 31/35 in folder AF23
Finished processing AF23 folder
Processing 35 images in 'AM24' folder...
  Processing image 1/35 in folder AM24
  Processing image 6/35 in folder AM24
  Processing image 11/35 in folder AM24
  Processing image 16/35 in folder AM24
  Processing image 21/35 in folder AM24
  Processing image 26/35 in folder AM24
  Processing image 31/35 in folder AM24
Finished processing AM24 folder
Processing 35 images in 'BF09' folder...
  Processing image 1/35 in folder BF09
  Processing image 6/35 in folder BF09
  Processing image 11/35 in folder BF09
  Processing image 16/35 in folder BF09
  Processing image 21/35 in folder BF09
  Processing image 26/35 in folder BF09
  Processing image 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>